# SRAG (SARS) data exploration from Brazil's OpenDataSUS

This notebook is meant to explore and understand the data contained in the SRAG data from Brazil's OpenDataSUS data. This data is available at [the OpenDataSUS website](https://opendatasus.saude.gov.br/dataset/bd-srag-2020).

The data is divided into the 2020 data and the earlier data. The first step is to download the data and consume it into `pandas` dataframes.

To download the data, we use the bash script `GET_DATA.sh`, which contains the following code, that downloads and checks the data for the project:

In [5]:
!pygmentize GET_DATA.sh

#!/bin/bash

BLUE="\e"
WHITE="\e"
RED="\e"
GREEN="\e"

MESSAGE="${BLUE} -> "
ERROR="${RED}ERROR: ${WHITE}"
SUCCESS="${GREEN}"

# This scripts gets all the data from the SRAG databases from Brazil's OpenDataSUS. The script verifies if the files exist and verifies the SHA256 sums.
echo
echo -e "${MESSAGE}Creating data directory..."
echo 
mkdir -p data

sums=(`cat sums | awk '{print $1}'`)

# The 2020 data constantly moves between adresses, since it's updated every few days, containing the new data. To get the data
# we use some grep magic
link2020=`curl -kL https://opendatasus.saude.gov.br/dataset/bd-srag-2020 | grep -o "https:.*\.csv"`

links=("https://opendatasus.saude.gov.br/dataset/18254c56-0859-4073-a6ea-977c8b85bd28/resource/861e8067-a23d-49f6-93a7-9b83c2f802ac/download/influd14_limpo-final.csv" "https://opendatasus.saude.gov.br/dataset/18254c56-0859-4073-a6ea-977c8b85bd28/resource/e5f0102f-a2e5-41d9-ac05-88e438c578ce/download/influd15_limpo-final.csv" "https://opendatasus.saude.go

In [4]:
!bash GET_DATA.sh



 -> Creating data directory...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23031  100 23031    0     0  20975      0  0:00:01  0:00:01 --:--:-- 20994
 -> Downloading the 2014 data...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5749k    0 5749k    0     0  1465k      0 --:--:--  0:00:03 --:--:-- 1465k


 -> Checking sha256 sum for 2014 file...


Successfully downloaded files for year 2014!


 -> Downloading the 2015 data...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4414k    0 4414k    0     0  1465k      0 --:--:--  0:00:03 --:--:-- 1465k


 -> Checking sha256 sum for 2015 file...


Successfully downloaded files for year 2015!


 -> 

After downloading the data, the next step is to consume it into 

In [46]:
import pandas as pd
%matplotlib inline

In [54]:
df_2020 = pd.read_csv('data/2020.csv', encoding='latin2', sep=';', low_memory=False)

In [53]:
df_2019=pd.read_csv('data/2019.csv', encoding='latin2', sep=';', low_memory=False)

In [68]:

df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326193 entries, 0 to 326192
Columns: 134 entries, DT_NOTIFIC to PAC_DSCBO
dtypes: float64(76), int64(10), object(48)
memory usage: 333.5+ MB


['DT_NOTIFIC',
 'SEM_NOT',
 'DT_SIN_PRI',
 'SEM_PRI',
 'SG_UF_NOT',
 'ID_REGIONA',
 'CO_REGIONA',
 'ID_MUNICIP',
 'CO_MUN_NOT',
 'ID_UNIDADE',
 'CO_UNI_NOT',
 'CS_SEXO',
 'DT_NASC',
 'NU_IDADE_N',
 'TP_IDADE',
 'COD_IDADE',
 'CS_GESTANT',
 'CS_RACA',
 'CS_ETINIA',
 'CS_ESCOL_N',
 'ID_PAIS',
 'CO_PAIS',
 'SG_UF',
 'ID_RG_RESI',
 'CO_RG_RESI',
 'ID_MN_RESI',
 'CO_MUN_RES',
 'CS_ZONA',
 'SURTO_SG',
 'NOSOCOMIAL',
 'AVE_SUINO',
 'FEBRE',
 'TOSSE',
 'GARGANTA',
 'DISPNEIA',
 'DESC_RESP',
 'SATURACAO',
 'DIARREIA',
 'VOMITO',
 'OUTRO_SIN',
 'OUTRO_DES',
 'PUERPERA',
 'CARDIOPATI',
 'HEMATOLOGI',
 'SIND_DOWN',
 'HEPATICA',
 'ASMA',
 'DIABETES',
 'NEUROLOGIC',
 'PNEUMOPATI',
 'IMUNODEPRE',
 'RENAL',
 'OBESIDADE',
 'OBES_IMC',
 'OUT_MORBI',
 'MORB_DESC',
 'VACINA',
 'DT_UT_DOSE',
 'MAE_VAC',
 'DT_VAC_MAE',
 'M_AMAMENTA',
 'DT_DOSEUNI',
 'DT_1_DOSE',
 'DT_2_DOSE',
 'ANTIVIRAL',
 'TP_ANTIVIR',
 'OUT_ANTIV',
 'DT_ANTIVIR',
 'HOSPITAL',
 'DT_INTERNA',
 'SG_UF_INTE',
 'ID_RG_INTE',
 'CO_RG_INTE',
 '

In [ ]:
import boto3
from sagemaker import get_execution_role